<a href="https://colab.research.google.com/github/samruddhi-tonghale/Helmet-Detection-for-Two-Wheeler-Riders/blob/main/Helmet_and_Numberplate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install Pillow
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install easyocr

from ultralytics import YOLO
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import pytesseract
import os
import datetime
import matplotlib.pyplot as plt
import easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 23.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from google.colab import drive
drive.mount('/content/drive')

import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import shutil

# Existing convert_voc_to_yolo function - refactored to accept list of xml paths
def convert_voc_to_yolo(xml_file_paths, output_label_folder, classes):
    os.makedirs(output_label_folder, exist_ok=True)
    for xml_path in xml_file_paths:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Get image dimensions from XML (preferred for VOC datasets)
        try:
            w = int(root.find("size/width").text)
            h = int(root.find("size/height").text)
        except AttributeError:
            print(f"Error: Could not find size/width or size/height in {xml_path}. Skipping.")
            continue

        out_file = os.path.join(output_label_folder, os.path.basename(xml_path).replace(".xml", ".txt"))
        with open(out_file, "w") as f:
            for obj in root.findall("object"):
                name = obj.find("name").text.strip()
                if name not in classes:
                    continue
                cls = classes.index(name)
                b = obj.find("bndbox")
                xmin, ymin, xmax, ymax = [float(b.find(t).text) for t in ("xmin","ymin","xmax","ymax")]
                x_center = ((xmin + xmax) / 2) / w
                y_center = ((ymin + ymax) / 2) / h
                bw = (xmax - xmin) / w
                bh = (ymax - ymin) / h
                f.write(f"{cls} {x_center} {y_center} {bw} {bh}\n")

def prepare_dataset(dataset_name, base_dir, annotation_folder, image_source_folder_name, classes, test_size=0.2):
    print(f"Preparing {dataset_name} dataset...")

    full_dataset_path = os.path.join(base_dir, dataset_name)
    raw_images_path = os.path.join(full_dataset_path, image_source_folder_name)
    raw_annotations_path = os.path.join(full_dataset_path, annotation_folder)

    # Define target directories for YOLO format
    train_images_target_path = os.path.join(full_dataset_path, "train", "images")
    val_images_target_path = os.path.join(full_dataset_path, "val", "images")
    train_labels_target_path = os.path.join(full_dataset_path, "train", "labels")
    val_labels_target_path = os.path.join(full_dataset_path, "val", "labels")

    # Create target directories
    os.makedirs(train_images_target_path, exist_ok=True)
    os.makedirs(val_images_target_path, exist_ok=True)
    os.makedirs(train_labels_target_path, exist_ok=True)
    os.makedirs(val_labels_target_path, exist_ok=True)

    # Get all image files and their corresponding XML file paths
    image_files = [f for f in os.listdir(raw_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Store tuples of (image_filename_without_extension, image_full_path, xml_full_path)
    data_items = []
    for img_file in image_files:
        img_name_without_ext = os.path.splitext(img_file)[0]
        xml_file = img_name_without_ext + ".xml"
        xml_full_path = os.path.join(raw_annotations_path, xml_file)
        if os.path.exists(xml_full_path):
            data_items.append((img_name_without_ext, os.path.join(raw_images_path, img_file), xml_full_path))
        else:
            print(f"  Warning: No XML found for image {img_file}. Skipping.")

    if not data_items:
        print(f"  No valid image-annotation pairs found in {dataset_name}. Skipping data preparation.")
        return

    # Split data_items into train and validation sets
    train_items, val_items = train_test_split(data_items, test_size=test_size, random_state=42)

    print(f"  Total items (image-XML pairs): {len(data_items)}")
    print(f"  Train items: {len(train_items)}")
    print(f"  Val items: {len(val_items)}")

    # Copy images and generate labels for training set
    train_xml_paths = []
    for img_name_no_ext, img_src_path, xml_src_path in train_items:
        shutil.copy(img_src_path, train_images_target_path)
        train_xml_paths.append(xml_src_path)
    convert_voc_to_yolo(train_xml_paths, train_labels_target_path, classes)

    # Copy images and generate labels for validation set
    val_xml_paths = []
    for img_name_no_ext, img_src_path, xml_src_path in val_items:
        shutil.copy(img_src_path, val_images_target_path)
        val_xml_paths.append(xml_src_path)
    convert_voc_to_yolo(val_xml_paths, val_labels_target_path, classes)

    print(f"Finished preparing {dataset_name}.")


# Base directory where datasets are located
base_drive_path = "/content/drive/MyDrive"

# ---------------- HELMET DATASET ----------------
prepare_dataset(
    dataset_name="HelmetDataset",
    base_dir=base_drive_path,
    annotation_folder="annotations",
    image_source_folder_name="images", # Assuming all raw images are in HelmetDataset/images
    classes=["with_helmet", "without_helmet"]
)

# ---------------- NUMBER PLATE DATASET ----------------
prepare_dataset(
    dataset_name="NumberplateDataset",
    base_dir=base_drive_path,
    annotation_folder="Annotations", # Case-sensitive for Numberplate
    image_source_folder_name="images", # Assuming all raw images are in NumberplateDataset/images
    classes=["license_plate"]
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preparing HelmetDataset dataset...
  Total items (image-XML pairs): 247
  Train items: 197
  Val items: 50
Finished preparing HelmetDataset.
Preparing NumberplateDataset dataset...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NumberplateDataset/images'

In [10]:
!ls "/content/drive/MyDrive/HelmetDataset/labels/train" | head
!ls "/content/drive/MyDrive/NumberplateDataset/labels/train" | head

!yolo train model=yolov8n.pt data="/content/drive/MyDrive/HelmetDataset.yaml" epochs=50 imgsz=640
!yolo train model=yolov8n.pt data="/content/drive/MyDrive/NumberplateDataset.yaml" epochs=50 imgsz=640

BikesHelmets0.txt
BikesHelmets100.txt
BikesHelmets101.txt
BikesHelmets102.txt
BikesHelmets103.txt
BikesHelmets104.txt
BikesHelmets105.txt
BikesHelmets106.txt
BikesHelmets107.txt
BikesHelmets108.txt
Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/HelmetDataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=tr

In [13]:
yaml_content = """
train: /content/drive/MyDrive/HelmetDataset/train/images
val: /content/drive/MyDrive/HelmetDataset/val/images

nc: 2
names: ['with_helmet', 'without_helmet']
"""

with open("/content/drive/MyDrive/HelmetDataset.yaml", "w") as f:
    f.write(yaml_content)
print("✅ HelmetDataset.yaml file created successfully with updated image paths!")


yaml_content = """
train: /content/drive/MyDrive/NumberplateDataset/train/images
val: /content/drive/MyDrive/NumberplateDataset/val/images

nc: 1
names: ['license_plate']
"""

with open("/content/drive/MyDrive/NumberplateDataset.yaml", "w") as f:
    f.write(yaml_content)
print("✅ NumberplateDataset.yaml file created successfully with updated image paths!")

✅ HelmetDataset.yaml file created successfully with updated image paths!
✅ NumberplateDataset.yaml file created successfully with updated image paths!


In [14]:
from ultralytics import YOLO
from IPython.display import Image, display
import glob
import os

# Load your trained YOLO model
# Assuming the last training run for HelmetDataset was 'train5' (from previous output)
model = YOLO("/content/runs/detect/train5/weights/best.pt", task="detect")

# Find .png and .jpg test images in your validation folder
test_images = glob.glob("/content/drive/MyDrive/HelmetDataset/val/images/*.png") + \
              glob.glob("/content/drive/MyDrive/HelmetDataset/val/images/*.jpg")

if len(test_images) == 0:
    print("⚠️ No images found in validation folder! Please check your path and image types.")
else:
    for img_path in test_images:
      print(f"Processing: {img_path}")

      # Run prediction
      results = model.predict(source=img_path, conf=0.5, save=True, show=False)

      # Locate the saved image
      # The results.save_dir points to the directory where images with predictions are saved.
      # It creates a new folder for each prediction run (e.g., 'predict', 'predict2', etc.)
      output_dir = results[0].save_dir
      saved_img = os.path.join(output_dir, os.path.basename(img_path))

      # Display the result
      display(Image(filename=saved_img))

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train5/weights/best.pt'